# Assignment 3: Sequence-to-Sequence Modeling (TA: Shivashish Naithani)

In this assignment, you will perform the task of translating Indian Names to English, a sequence-to-sequence modeling task, using character-level conditional language models.

As before, please make a copy of this notebook (locally or on Colab). Ensure you adhere to the guidelines and submission instructions (mentioned below) for attempting and submitting the assignment.

### Guidelines for Attempting the Assignment

1. Write your logic in the cells which have the comment `# ADD YOUR CODE HERE`, between the `# BEGIN CODE` and `# END CODE` comments. These cells are also demarcated by the special start (`## ==== BEGIN EVALUATION PORTION`) and end (`## ==== END EVALUATION PORTION`) comments. Do **NOT** remove any of these comments from the designated cells, otherwise your assigment may not be evaluated correctly.

2. All imports that should be necessary are already provided as part of the notebook. Should you require additional imports, add them in the cells to be graded, but outside the `# BEGIN CODE` and `# END CODE` block. For example, if you need to import a package called `mypackage`, add it as follows in a graded cell:

``` python
## ==== BEGIN EVALUATION PORTION

import mypackage # <===

def function_to_be_implemented(*args, **kwargs):

    ...

    # ADD YOUR CODE HERE
    # BEGIN CODE

    # END CODE

    ...

## ==== END EVALUATION PORTION

```
4. Do not modify anything in the cells which start with `# Please do not change anything in the following cell`.

5. Only write your code in the cells designated for auto-evaluation. If you encounter any errors in the supporting cells during execution, contact the respective TAs.

6. **Important**: Use of AI-assistive technologies such as ChatGPT or GitHub CoPilot is not permitted for this assignment. Ensure that all attempts are solely your own. Not following this rule can incur heavy penalty, including getting NO GRADE for this assignment, which will affect your grade significantly.

### Submission Instructions

1. Ensure your code follows all guidelines mentioned above before submission.

2. Try to avoid any unnecessary print statements across the code. We will evaluate specific output lines which begin with the phrase `EVALUATION`. Ensure you do not modify these print statements howsoever, as they are used for auto-evaluation.

3. When you have completely attempted the assignment, export the current notebook as a `.py` file, with the following name: `SAPName_SRNo_assignment3.py`, where `SAPName` would be your name as per SAP record, and `SRNo` will be the last 5 digits of your IISc SR number. For example, IISc student with SAP name Twyla Linda (SR no - 04-03-00-10-22-20-1-15329) would use `Twyla_Linda_15329_assignment3.py`.

4. Once you have executed the code, certain additional files will be created. Once you are done executing all associated cells, ensure the folder structure looks as follows:

``` python
└─── SAPName_SRNo
     ├─── SAPName_SRNo_assignment3.py
     ├─── src-tokenizer
     │    └─── tokenizer.pkl
     ├─── tgt-tokenizer
     │    └─── tokenizer.pkl
     ├─── rnn.enc-dec
     │    ├─── model.pt
     │    ├─── loss.json
     │    ├─── outputs.csv
     │    └─── metadata.json
     └─── rnn.enc-dec.attn
          ├─── model.pt
          ├─── loss.json
          ├─── outputs.csv
          └─── metadata.json
```

5. Once you have validated the folder structure as above, add the exported `.py` file to the folder and submit the folder as a ZIP archive.

In the cell below, replace `SAPName` with your name as per SAP record, and `SRNo` with the last 5 digits of your IISc SR number. For example, IISc student with SAP name Twyla Linda (SR no - 04-03-00-10-22-20-1-15329) would use:

```python
STUDENT_SAP_NAME  = "Twyla Linda"
STUDENT_SR_NUMBER = "15329"
```

In [1]:
STUDENT_SAP_NAME  = "Godwin_Khalko"
STUDENT_SR_NUMBER = "24006"

**Important Notes**:

- Some of the tasks in this assignment are compute intensive, and are better performed on an accelerator device (GPU, etc.). Unless you have one locally, prefer using a GPU instance on Colab for execution.
- Due to resource restrictions on Colab, training some models may not finish in time. In such a case, ensure you store checkpoints to a persistent directory so that you may resume training once your resource limits are restored.

## Outline

Through the last assignment, you have seen that neural language models are able to successfully capture patterns across Indian names. In this assignment, you will extend upon that idea to learn conditional language models for the task of transliteration: converting Indian names in the English alphabet to Hindi.

### Marks Distribution

- Agnostic Task-Specific Training: 5 marks
- Seq-2-Seq via RNN: 40 marks
- Seq-2-Seq via RNN with Attention: 35 marks
- Evaluation
- Decoding Strategy: 20 marks

## Setup

The following cells perform the basic setup such as importing the necessary packages. You will not require any additional libraries, so importing any additional libraries is not allowed.

In [2]:
# Installs packages, if using locally. Feel free to add other missing packages as required.

%pip install tqdm nltk matplotlib numpy pandas

In [3]:
# Built-in imports, no installations required.

import os
import gc
import json
import math
import pickle
import subprocess
import collections
import unicodedata

In [4]:
# 3rd-party package imports, may require installation if not on a platform such as Colab.

import numpy
import torch
import pandas as pd
import tqdm.auto as tqdm

import matplotlib
from matplotlib import pyplot
from nltk.translate import bleu_score
from torch.utils.data import TensorDataset, DataLoader

In [5]:
# Please do not change anything in the following cell

# Find and load fonts that can display Hindi characters, for Matplotlib
result = subprocess.run([ 'fc-list', ':lang=hi', 'family' ], capture_output=True)
found_hindi_fonts = result.stdout.decode('utf-8').strip().split('\n')

matplotlib.rcParams['font.sans-serif'] = [
    'Source Han Sans TW', 'sans-serif', 'Arial Unicode MS',
    *found_hindi_fonts
]

In [6]:
# Please do not change anything in the following cell

DIRECTORY_NAME = f"{STUDENT_SAP_NAME.replace(' ', '_')}_{STUDENT_SR_NUMBER}"

os.makedirs(DIRECTORY_NAME, exist_ok=True)

In [8]:
def sync_vram():
    """ Synchronizes the VRAM across the GPUs, reclaiming unused memory. """
    #TODO:Delte the if condition
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.synchronize()

## Data Preparation

We'll load the data for the task, which comprises of a parallel corpus of Indian Names and their Hindi equivalents.

In [9]:
# Make sure your code is not dependent on any of the file names as below.

# Download the training and validation datasets
!wget -O data.train.csv "https://docs.google.com/spreadsheets/d/e/2PACX-1vQOYe_Oy8eMzVFYq6hBSyPLslqA1PeMeK8S5nPs2-viuCNzx0i3Fl_ptFmD0YD3kTA_olYdOIx7iPOh/pub?gid=1482240395&single=true&output=csv"
!wget -O data.valid.csv "https://docs.google.com/spreadsheets/d/e/2PACX-1vTKa_jeysYhx869fmTb7VUchlSiChUq0vqotWRGMmnTXWZ8H2PkF8s6hRr2vdo6v54JJx8CEuVo8MZ3/pub?gid=1579594041&single=true&output=csv"

--2025-03-27 19:03:04--  https://docs.google.com/spreadsheets/d/e/2PACX-1vQOYe_Oy8eMzVFYq6hBSyPLslqA1PeMeK8S5nPs2-viuCNzx0i3Fl_ptFmD0YD3kTA_olYdOIx7iPOh/pub?gid=1482240395&single=true&output=csv
Resolving docs.google.com (docs.google.com)... 142.251.2.138, 142.251.2.102, 142.251.2.100, ...
Connecting to docs.google.com (docs.google.com)|142.251.2.138|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: https://doc-08-6c-sheets.googleusercontent.com/pub/54bogvaave6cua4cdnls17ksc4/hg8r4rpj2m3sojne5fats4f9k0/1743102185000/118252164104506406045/*/e@2PACX-1vQOYe_Oy8eMzVFYq6hBSyPLslqA1PeMeK8S5nPs2-viuCNzx0i3Fl_ptFmD0YD3kTA_olYdOIx7iPOh?gid=1482240395&single=true&output=csv [following]
--2025-03-27 19:03:05--  https://doc-08-6c-sheets.googleusercontent.com/pub/54bogvaave6cua4cdnls17ksc4/hg8r4rpj2m3sojne5fats4f9k0/1743102185000/118252164104506406045/*/e@2PACX-1vQOYe_Oy8eMzVFYq6hBSyPLslqA1PeMeK8S5nPs2-viuCNzx0i3Fl_ptFmD0YD3kTA_olYdOIx7iPOh?gid=1482240395&s

In [10]:
# Please do not change anything in the following cell

def read_dataframe(ds_type):
    """ Loads a dataframe based on the given partition type.

    Args:
        ds_type (str): Dataset type: train (train) or validation (valid)

    Returns:
        pd.DataFrame: Pandas Dataframe for the specified partition.
    """

    df = pd.read_csv(f"data.{ds_type}.csv", header=0)
    df = df[~df.isna()]
    df['Name'] = df['Name'].astype(str)
    df['Translation'] = df['Translation'].astype(str)
    return df

# Load the training and validation datasets
train_data      = read_dataframe("train")
validation_data = read_dataframe("valid")

print(f"Length of training data: {len(train_data)}\nLength of validation data: {len(validation_data)}")

Length of training data: 4484
Length of validation data: 400


Here are some examples from the training dataset. Note that the dataset may be noisy so some examples may not be perfect:

In [11]:
train_data.sample(n=5)

,Name,Translation
1038,काशीराम,kashiram
4058,सोहनलाल,sohanlal
221,सहजा,sahaja
2369,भानुप्रिया,bhanupriya
2284,सलेश,salesh


## Tokenization

An implementation of tokenization is already in place. Do not modify it, as it can affect your evaluation.

In [12]:
# Please do not change anything in the following cell

class Tokenizer:
    """ Represents the tokenizer for text data.
        Provides methods to encode and decode strings (as instance or as a batch). """

    def __init__(self):
        """ Initializes a new tokenizer.

            Any variables required in intermediate operations are declared here.
            You will also need to define things like special tokens and other things here.

            All variables declared in this function will be serialized
                and deserialized when loading and saving the Tokenizer.
            """

        self.special_tokens = { '[BOS]': 1, '[EOS]': 2, '[PAD]': 0 }
        self.vocab = { bytes([ i ]): i+len(self.special_tokens) for i in range(256)  }
        self.merge_rules = {  }
        self.inv_vocab = { _id: token for token, _id in self.vocab.items() }
        self.inv_vocab.update({ _id: token.encode() for token, _id in self.special_tokens.items() })

    @classmethod
    def load(cls, path):
        """ Loads a pre-trained tokenizer from the given directory.
           This directory will have a tokenizer.pkl file that contains all the tokenizer variables.

        Args:
            path (str): Path to load the tokenizer from.
        """
        tokenizer_file = os.path.join(path, "tokenizer.pkl")

        if not os.path.exists(path) or not os.path.exists(os.path.join(path, "tokenizer.pkl")):
            raise ValueError(cls.load.__name__ + ": No tokenizer found at the specified directory")

        with open(tokenizer_file, "rb") as ifile:
            return pickle.load(ifile)

    def save(self, path):
        """ Saves a trained tokenizer to a given directory, inside a tokenizer.pkl file.

        Args:
            path (str): Directory to save the tokenizer in.
        """

        os.makedirs(path, exist_ok=True)
        with open(os.path.join(path, "tokenizer.pkl"), 'wb') as ofile:
            pickle.dump(self, ofile)

    def train(self, data, vocab_size):
        """ Trains a tokenizer to learn meaningful representations from input data.
            In the end, learns a vocabulary of a fixed size over the given data.
            Special tokens, if any, must not be counted towards this vocabulary.

        Args:
            data (list[str]): List of input strings from a text corpus.
            vocab_size (int): Final desired size of the vocab to be learnt.
        """

        self.vocab = { bytes([ i ]): i+len(self.special_tokens) for i in range(256)  }
        self.vocab.update({ token.encode('utf-8'): _id for token, _id in self.special_tokens.items() })

        self.merge_rules = {  }
        self.inv_vocab   = { _id: token for token, _id in self.vocab.items() }

        data = [ [ i+len(self.special_tokens) for i in instance.encode('utf-8') ] for instance in data ]

        while len(self.vocab) < len(self.special_tokens) + vocab_size:
            # Compute stats
            counts = collections.defaultdict(int)
            for tok_str in data:
                for tok, next_tok in zip(tok_str, tok_str[1:]):
                    counts[(tok, next_tok)] += 1

            # Learn a new merge rule
            best_pair = max(counts, key=counts.get)
            new_token, new_id = self.inv_vocab[best_pair[0]] + self.inv_vocab[best_pair[1]], len(self.vocab)
            self.merge_rules[best_pair] = new_id
            self.inv_vocab[new_id] = new_token
            self.vocab[new_token]  = new_id

            # Update tokens
            new_data = []
            for tok_str in data:
                i, new_tok_str = 0, []
                while i < len(tok_str):
                    if i < len(tok_str) - 1 and (tok_str[i], tok_str[i+1]) == best_pair:
                        new_tok_str.append(new_id)
                        i += 2
                    else:
                        new_tok_str.append(tok_str[i])
                        i += 1
                new_data.append(new_tok_str)
            data = new_data

    def pad(self, tokens, length):
        """ Pads a tokenized string to a specified length, for batch processing.

        Args:
            tokens (list[int]): Encoded token string to be padded.
            length (int): Length of tokens to pad to.

        Returns:
            list[int]: Token string padded to desired length.
        """


        if len(tokens) < length:
            tokens = [ *tokens ]
            tokens += ([ self.special_tokens['[PAD]'] ] * (length - len(tokens)))

        return tokens


    def unpad(self, tokens):
        """ Removes padding from a token string.

        Args:
            tokens (list[int]): Encoded token string with padding.

        Returns:
            list[int]: Token string with padding removed.
        """

        no_pad_len = len(tokens)
        while tokens[no_pad_len-1] == self.special_tokens['[PAD]']: no_pad_len -= 1

        return tokens[:no_pad_len]

    def get_special_tokens(self):
        """ Returns the associated special tokens.

            Returns:
                dict[str, int]: Mapping describing the special tokens, if any.
                    This is a mapping between a string segment (token) and its associated id (token_id).
        """

        return self.special_tokens

    def get_vocabulary(self):
        """ Returns the learnt vocabulary post the training process.

            Returns:
                dict[str, int]: Mapping describing the vocabulary and special tokens, if any.
                    This is a mapping between a string segment (token) and its associated id (token_id).
        """

        return self.vocab

    def encode(self, string, add_start=True, add_end=True):
        """ Encodes a string into a list of tokens.

        Args:
            string (str): Input string to be tokenized.
            add_start (bool): If true, adds the start of sequence token.
            add_end (bool): If true, adds the end of sequence token.
        Returns:
            list[int]: List of tokens (unpadded).
        """

        string = unicodedata.normalize('NFKC', string)

        tokens = [ i+len(self.special_tokens) for i in string.encode('utf-8') ]

        while len(tokens) > 1:
            pairs = set()
            for tok, next_tok in zip(tokens, tokens[1:]):
                pairs.add((tok, next_tok))

            merge_pair = min(pairs, key=lambda x: self.merge_rules.get(x, float("inf")))
            if merge_pair not in self.merge_rules: break

            i, new_tokens = 0, []
            while i < len(tokens):
                if i < len(tokens) - 1 and (tokens[i], tokens[i+1]) == merge_pair:
                    new_tokens.append(self.merge_rules[merge_pair])
                    i += 2
                else:
                    new_tokens.append(tokens[i])
                    i += 1
            tokens = new_tokens

        if add_start: tokens = [ self.special_tokens['[BOS]'] ] + tokens
        if add_end  : tokens = tokens + [ self.special_tokens['[EOS]'] ]

        return tokens


    def decode(self, tokens, strip_special=True):
        """ Decodes a string from a list of tokens.
            Undoes the tokenization, returning back the input string.

        Args:
            tokens (list[int]): List of encoded tokens to be decoded. No padding is assumed.
            strip_special (bool): Whether to remove special tokens or not.

        Returns:
            str: Decoded string.
        """

        if strip_special:
            special_tokens = set(self.special_tokens.values())
            tokens = [ token for token in tokens if token not in special_tokens ]

        return (b''.join(self.inv_vocab[tok_id] for tok_id in tokens)).decode('utf-8', errors='replace')

    def batch_encode(self, batch, padding=None, add_start=True, add_end=True):
        """Encodes multiple strings in a batch to list of tokens padded to a given size.

        Args:
            batch (list[str]): List of strings to be tokenized.
            padding (int, optional): Optional, desired tokenized length. Outputs will be padded to fit this length.
            add_start (bool): If true, adds the start of sequence token.
            add_end (bool): If true, adds the end of sequence token.

        Returns:
            list[list[int]]: List of tokenized outputs, padded to the same length.
        """

        batch_output = [ self.encode(string, add_start, add_end) for string in batch ]
        if padding:
            for i, tokens in enumerate(batch_output):
                if len(tokens) < padding:
                    batch_output[i] = self.pad(tokens, padding)
        return batch_output

    def batch_decode(self, batch, strip_special=True):
        """ Decodes a batch of encoded tokens to normal strings.

        Args:
            batch (list[list[int]]): List of encoded token strings, optionally padded.
            strip_special (bool): Whether to remove special tokens or not.

        Returns:
            list[str]: Decoded strings after padding is removed.
        """
        return [ self.decode(self.unpad(tokens), strip_special=strip_special) for tokens in batch ]

Now with the tokenizer class, initialize and train the tokenizers for processing the parallel corpus:

In [13]:
# Please do not change anything in the following cell

# Initialize the tokenizers as per the desired strategy.
src_tokenizer = Tokenizer()
tgt_tokenizer = Tokenizer()


In [14]:
# Please do not change anything in the following cell

# Edit the hyperparameters below as desired.
SRC_VOCAB_SIZE = 300
TGT_VOCAB_SIZE = 400


# Train your tokenizer(s)
src_tokenizer.train(train_data['Name']       , vocab_size=SRC_VOCAB_SIZE)
tgt_tokenizer.train(train_data['Translation'], vocab_size=TGT_VOCAB_SIZE)

In [15]:
# Please do not change anything in the following cell

# Save the trained tokenizers
src_tokenizer.save(os.path.join(DIRECTORY_NAME, "src-tokenizer"))
tgt_tokenizer.save(os.path.join(DIRECTORY_NAME, "tgt-tokenizer"))

In [16]:
# Please do not change anything in the following cell

def render_glyph(token):
    """ Renders a token, handling invalid bytes in a safe, error-proof manner. """

    token = token.decode('utf-8', errors='replace') if isinstance(token, bytes) else token
    return "".join([ c if unicodedata.category(c)[0] != "C" else f"\\u{ord(c):04x}" for c in token ])

def inverse_vocabulary(tokenizer):
    """ Generates an inverse vocabulary with rendered tokens.

    Args:
        tokenizer (Tokenizer): Tokenizer whose vocabulary must be used.
    """

    return { id: render_glyph(token) for token, id in tokenizer.get_vocabulary().items() }

def apply_inverse_vocab(tokens, inv_vocab):
    """ Decodes using the given inverse vocabulary.

    Args:
        tokens (list[int]): Tokens to process.
        inv_vocab (dict[int, str]): Inverse vocabulary for mapping ids to tokens.

    Returns:
        list[str]: Mapped token glyphs.
    """

    return [ inv_vocab[id] for id in tokens ]

We visualize a few outputs of the learnt tokenizers to assess their working:

In [17]:
# Please do not change anything in the following cell

src_id_to_token = inverse_vocabulary(src_tokenizer)

for example in train_data['Name'].sample(n=5, random_state=20240227):
    print("Name           :", example)
    tokens = src_tokenizer.encode(example)
    print("Tokens         :", tokens)
    print("Tokens (glyphs):", apply_inverse_vocab(tokens, src_id_to_token))
    print("Decoded        :", src_tokenizer.decode(tokens), end='\n\n')

Name           : दीपक
Tokens         : [1, 274, 265, 278, 273, 2]
Tokens (glyphs): ['[BOS]', 'द', 'ी', 'प', 'क', '[EOS]']
Decoded        : दीपक

Name           : दामोदर
Tokens         : [1, 274, 261, 264, 282, 274, 262, 2]
Tokens (glyphs): ['[BOS]', 'द', 'ा', 'म', 'ो', 'द', 'र', '[EOS]']
Decoded        : दामोदर

Name           : आमरिन
Tokens         : [1, 300, 264, 262, 267, 263, 2]
Tokens (glyphs): ['[BOS]', 'आ', 'म', 'र', 'ि', 'न', '[EOS]']
Decoded        : आमरिन

Name           : बनिता
Tokens         : [1, 279, 263, 267, 297, 2]
Tokens (glyphs): ['[BOS]', 'ब', 'न', 'ि', 'ता', '[EOS]']
Decoded        : बनिता

Name           : घन्स्याम
Tokens         : [1, 259, 155, 263, 266, 268, 266, 291, 264, 2]
Tokens (glyphs): ['[BOS]', '�', '�', 'न', '्', 'स', '्', 'या', 'म', '[EOS]']
Decoded        : घन्स्याम



In [18]:
# Please do not change anything in the following cell

tgt_id_to_token = inverse_vocabulary(tgt_tokenizer)

for example in train_data['Translation'].sample(n=5, random_state=20240227):
    print("Name           :", example)
    tokens = tgt_tokenizer.encode(example)
    print("Tokens         :", tokens)
    print("Tokens (glyphs):", apply_inverse_vocab(tokens, tgt_id_to_token))
    print("Decoded        :", tgt_tokenizer.decode(tokens), end='\n\n')

Name           : deepak
Tokens         : [1, 332, 318, 110, 2]
Tokens (glyphs): ['[BOS]', 'dee', 'pa', 'k', '[EOS]']
Decoded        : deepak

Name           : damodar
Tokens         : [1, 103, 264, 114, 103, 261, 2]
Tokens (glyphs): ['[BOS]', 'd', 'am', 'o', 'd', 'ar', '[EOS]']
Decoded        : damodar

Name           : aamrin
Tokens         : [1, 100, 264, 117, 265, 2]
Tokens (glyphs): ['[BOS]', 'a', 'am', 'r', 'in', '[EOS]']
Decoded        : aamrin

Name           : banita
Tokens         : [1, 101, 259, 327, 2]
Tokens (glyphs): ['[BOS]', 'b', 'an', 'ita', '[EOS]']
Decoded        : banita

Name           : ghansyam
Tokens         : [1, 106, 345, 118, 375, 2]
Tokens (glyphs): ['[BOS]', 'g', 'han', 's', 'yam', '[EOS]']
Decoded        : ghansyam



We now abstract away the tokenizer into a pytorch compatible TokenizedDataset that will handle the tokenization internally:

In [19]:
# Please do not change anything in the following cell

class TokenizerDataset(TensorDataset):
    """ Abstraction of the tokenizer functions as a pytorch dataset. """

    def __init__(self, data, src_tokenizer, tgt_tokenizer, src_padding=None, tgt_padding=None):
        """ Initializes the dataset.

        Args:
            data: DataFrame of input and output strings.
            src_tokenizer (Tokenizer): Tokenizer for the source language.
            tgt_tokenizer (Tokenizer): Tokenizer for the target language.
            src_padding (int, optional): Padding length for the source text. Defaults to None.
            tgt_padding (int, optional): Padding length for the target text. Defaults to None.
        """

        self.data = data
        self.src_tokenizer = src_tokenizer
        self.tgt_tokenizer = tgt_tokenizer
        self.src_padding = src_padding
        self.tgt_padding = tgt_padding

    def collate(self, batch):
        """ Collates data instances into a batch of tokenized tensors.

        Args:
            batch (list[tuple]): List of x, y pairs.

        Returns:
            tuple[torch.Tensor|PackedSequence, torch.Tensor|PackedSequence]: pair of tokenized tensors.
        """

        x_batch = [ data[0] for data in batch ]
        y_batch = [ data[1] for data in batch ]

        x_batch = self.src_tokenizer.batch_encode(x_batch, self.src_padding)
        y_batch = self.tgt_tokenizer.batch_encode(y_batch, self.tgt_padding)

        if self.src_padding is None:
            x_batch = torch.nn.utils.rnn.pack_sequence([ torch.tensor(tokens) for tokens in x_batch ], False)
        else:
            x_batch = torch.tensor(x_batch)

        if self.tgt_padding is None:
            y_batch = torch.nn.utils.rnn.pack_sequence([ torch.tensor(tokens) for tokens in y_batch ], False)
        else:
            y_batch = torch.tensor(y_batch)

        return x_batch, y_batch

    def __getitem__(self, index):
        """ Returns the nth instance from the dataset.

        Args:
            index (int): Index of the instance to retrieve.

        Returns:
            tuple[str, str]: Untokenized instance pair.
        """

        return (
            self.data['Name'][index],
            self.data['Translation'][index]
        )

    def __len__(self):
        """ Returns the length of the dataset.

        Returns:
            int: Length of the dataset.
        """
        return len(self.data)

## Model-Agnostic Training

Next, you'll implement a Trainer to train different models, since the data and tokenizer remains the same for all models.

This trainer will receive the model, a loss function, an optimizer, a training and (optionally) a validation dataset and use these to train (and validate) the model.

The trainer will also take care of handling checkpoints for training, which can be used to resume training across sessions.

Derived classes can also be defined to handle different architectures, as to be done in the model-specific classes below.

In [20]:
## ==== BEGIN EVALUATION PORTION

class Trainer:
    """ Performs model training in a model-agnostic manner.
        Requires specifying the model instance, the loss criterion to optimize,
          the optimizer to use and the directory to save data to.
    """

    def __init__(self, directory, model, criterion, optimizer):
        """ Initializes the trainer.

        Args:
            directory (str): Directory to save checkpoints and the model data in.
            model (torch.nn.Module): Torch model (must inherit `torch.nn.Module`) to train.
            criterion (torch.nn.Function): Loss criterion, i.e., the loss function to optimize for training.
            optimizer (torch.optim.Optimizer): Optimizer to use for training.
        """

        self.model            = model
        self.optimizer        = optimizer
        self.criterion        = criterion
        self.directory        = directory
        self.last_checkpoint  = 0
        self.loss_history     = { 'train': [], 'valid': [] }

        os.makedirs(self.directory, exist_ok=True)
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

    @staticmethod
    def make_dataloader(dataset, shuffle_data=True, batch_size=8, collate_fn=None):
        """ Create a dataloader for a torch Dataset.

        Args:
            dataset (torch.utils.data.Dataset): Dataset to process.
            shuffle_data (bool, optional): If true, shuffles the data. Defaults to True.
            batch_size (int, optional): Number of items per batch. Defaults to 8.
            collate_fn (function, optional): Function to use for collating instances to a batch.

        Returns:
            torch.utils.data.DataLoader: Dataloader over the given data, post processing.
        """

        # BEGIN CODE : trainer.make_dataloader

        # ADD YOUR CODE HERE
        dataloader = torch.utils.data.DataLoader(dataset = dataset, batch_size=batch_size, shuffle=shuffle_data, collate_fn=collate_fn)
        return dataloader

        # END CODE

    def train_step(self, x_batch, y_batch):
        """ Performs a step of training, on the training batch.

        Args:
            x_batch (torch.Tensor): Input batch.
            y_batch (torch.Tensor): Output batch.

        Returns:
            float: Training loss with the current model, on this batch.
        """

        # BEGIN CODE : trainer.train_step

        # ADD YOUR CODE HERE
        # First set the model to train mode
        self.model.train()


        x_batch = x_batch.to(self.device)
        y_batch = y_batch.to(self.device)
        self.model = self.model.to(self.device)

        #Generate the outputs
        self.optimizer.zero_grad()
        output = self.model(x_batch)

        #Calculate the loss
        loss = self.criterion(output, y_batch)

        #Start the back-propagration
        loss.backward()

        #Update the weights
        self.optimizer.step()

        #print and return the loss
        return loss.item()
        # END CODE

    def eval_step(self, validation_dataloader):
        """ Perfoms an evaluation step, on the validation dataloader.

        Args:
            validation_dataloader (torch.utils.data.DataLoader): Dataloader for the validation dataset.

        Returns:
            float: Validation loss with the current model checkpoint.
        """

        # BEGIN CODE : trainer.eval_step

        # ADD YOUR CODE HERE
        self.model.eval()
        val_loss_list = []
        with torch.no_grad():
          for x_val, y_val in validation_dataloader:

            #Move the tensors to device
            x_val = x_val.to(self.device)
            y_val = y_val.to(self.device)

            #Calcualte the outputs
            val_output = self.model(x_val)

            #Calculate the validation loss
            val_loss = self.criterion(val_output, y_val)

            #Append the validaiton loss to the list
            val_loss_list.append(val_loss.item())

        avg_val_loss = numpy.mean(val_loss_list)
        return avg_val_loss
        # END CODE

    def train(self, train_dataset, validation_dataset=None,
              num_epochs=10, batch_size=8, shuffle=True,
              save_steps=100, eval_steps=100, collate_fn=None):
        """ Handles the training loop for the model.

        Args:
            train_dataset (torch.utils.data.Dataset): Dataset to train on.
            validation_dataset (torch.utils.data.Dataset, optional): Data to validate on. Defaults to None.
            num_epochs (int, optional): Number of epochs to train for. Defaults to 10.
            batch_size (int, optional): Number of items to process per batch. Defaults to 8.
            shuffle (bool, optional): Whether to shuffle the data or not. Defaults to True.
            save_steps (int, optional): Number of steps post which a checkpoint should be saved. Defaults to 100.
            eval_steps (int, optional): Number of steps post which the model should be evaluated. Defaults to 100.
            collate_fn (function, optional): Function to use for collating instances to a batch.
        """

        current_checkpoint = 0
        self.model.to(self.device)
        self.model.train()

        with tqdm.tqdm(total = math.ceil(len(train_dataset) / batch_size) * num_epochs) as pbar:
            for epoch in range(num_epochs):
                train_dataloader      = self.make_dataloader(train_dataset, shuffle, batch_size, collate_fn)
                if validation_dataset is not None:
                    validation_dataloader = self.make_dataloader(validation_dataset, shuffle, batch_size, collate_fn)

                for batch, (x_batch, y_batch) in enumerate(train_dataloader):
                    pbar.set_description(f"Epoch {epoch+1} / {num_epochs}")

                    # If we are resuming training, skip this iteration
                    if current_checkpoint < self.last_checkpoint:
                        current_checkpoint += 1
                        pbar.update()
                        continue

                    # Do a step of training
                    loss = self.train_step(x_batch, y_batch)
                    self.loss_history['train'].append(loss)
                    pbar.set_postfix({ 'batch': batch+1, 'loss': loss })

                    current_checkpoint += 1
                    pbar.update()

                    # Evaluate after every eval_steps
                    if (current_checkpoint) % eval_steps == 0:
                        if validation_dataset is not None:
                            val_loss = self.eval_step(validation_dataloader)
                            self.loss_history['valid'].append(val_loss)
                        else:
                            val_loss = None

                        print('[>]', f"epoch #{epoch+1:{len(str(num_epochs))}},",
                              f"batch #{batch+1:{len(str(len(train_dataloader)))}}:",
                              "loss:", f"{loss:.8f}", '|', "val_loss:", f"{val_loss:.8f}")

                    # Save after every save_steps
                    if (current_checkpoint) % save_steps == 0:
                        self.save(current_checkpoint, { 'loss': loss, 'checkpoint': current_checkpoint })

                    # free unused resources
                    sync_vram()

            self.save(current_checkpoint)

    def resume(self):
        """ Resumes training session from the most recent checkpoint. """

        if checkpoints := os.listdir(self.directory):
            self.last_checkpoint = max(map(lambda x: int(x[11:]), filter(lambda x: 'checkpoint-' in x, checkpoints)))
            checkpoint_dir = os.path.join(self.directory, f"checkpoint-{self.last_checkpoint}")
            self.model.load_state_dict(torch.load(
                os.path.join(checkpoint_dir, "model.pt"),
                map_location=self.device
            ))
            self.model.to(self.device)
            self.optimizer.load_state_dict(torch.load(
                os.path.join(checkpoint_dir, "optimizer.pt"),
                map_location=self.device
            ))
            with open(os.path.join(checkpoint_dir, "loss.json"), 'r', encoding='utf-8') as ifile:
                self.loss_history = json.load(ifile)

    def save(self, checkpoint=None, metadata=None):
        """ Saves an associated model or a training checkpoint.

            If a checkpoint is specified, saves a checkpoint specific directory with optimizer data
                so that training can be resumed post that checkpoint.

        Args:
            checkpoint (int, optional): Checkpoint index. Defaults to None.
            metadata (dict[str, any], optional): Additional metadata to save alongside a checkpoint. Defaults to None.
        """

        if checkpoint is not None:
            checkpoint_dir = os.path.join(self.directory, f"checkpoint-{checkpoint}")
            os.makedirs(checkpoint_dir, exist_ok=True)
            torch.save(self.model.state_dict(), os.path.join(checkpoint_dir, "model.pt"))
            torch.save(self.optimizer.state_dict(), os.path.join(checkpoint_dir, "optimizer.pt"))
            with open(os.path.join(checkpoint_dir, "loss.json"), "w+", encoding='utf-8') as ofile:
                json.dump(self.loss_history, ofile, ensure_ascii=False, indent=2)
            if metadata:
                with open(os.path.join(checkpoint_dir, "metadata.json"), "w+", encoding='utf-8') as ofile:
                    json.dump(metadata, ofile, ensure_ascii=False, indent=2)
        else:
            torch.save(self.model, os.path.join(self.directory, "model.pt"))
            with open(os.path.join(self.directory, "loss.json"), "w+", encoding='utf-8') as ofile:
                json.dump(self.loss_history, ofile, ensure_ascii=False, indent=2)
            if metadata:
                with open(os.path.join(self.directory, "metadata.json"), "w+", encoding='utf-8') as ofile:
                    json.dump(metadata, ofile, ensure_ascii=False, indent=2)

## ==== END EVALUATION PORTION

To test that the trainer works, try training a simple MLP network:

In [21]:
X_train = torch.rand((500, 2))                      # (N x 2)
X_dev   = torch.rand((20 , 2))                      # (N x 2)

Y_train = (X_train[:, 0] - X_train[:, 1])[:, None]  # (N x 1)
Y_dev   = (X_dev  [:, 0] - X_dev  [:, 1])[:, None]  # (N x 1)

dummy_train_dataset = TensorDataset(X_train, Y_train)
dummy_val_dataset   = TensorDataset(X_dev  , Y_dev  )

In [22]:
model = torch.nn.Sequential(
    torch.nn.Linear(2, 4),
    torch.nn.ReLU(),
    torch.nn.Linear(4, 1)
)

loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

trainer = Trainer("mlp", model, loss_fn, optimizer)
trainer.train(dummy_train_dataset, dummy_val_dataset, batch_size=10, save_steps=300)

  0%|          | 0/500 [00:00<?, ?it/s]

[>] epoch # 2, batch #50: loss: 0.04540730 | val_loss: 0.06170079
[>] epoch # 4, batch #50: loss: 0.12230095 | val_loss: 0.04819358
[>] epoch # 6, batch #50: loss: 0.03422758 | val_loss: 0.03784056
[>] epoch # 8, batch #50: loss: 0.02580723 | val_loss: 0.02896213
[>] epoch #10, batch #50: loss: 0.05213849 | val_loss: 0.02171193


## Seq-2-Seq Modeling with RNNs

In this section, you will implement an encoder-decoder network using RNNs, to learn a conditional language model for the task of translating the names to Hindi.

You can use any type of RNN for this purpose: `RNN`, `GRU`, `LSTM`, etc. Consult the pytorch documentation for additional information.

Additional tips for training:
- Use regularization: Dropout, etc.
- Use a suitable optimizer, such as Adam.
- Format data accordingly before passing it to the trainer, using the helper functions.
- Do you need to pad sequences when processing inputs as a batch?

In [23]:
## ==== BEGIN EVALUATION PORTION

class RNNEncoderDecoderLM(torch.nn.Module):
    """ Implements an Encoder-Decoder network, using RNN units. """

    # Feel free to add additional parameters to __init__
    def __init__(self, src_vocab_size, tgt_vocab_size, embd_dims, hidden_size, num_layers=1, dropout=0.1):
        """ Initializes the encoder-decoder network, implemented via RNNs.

        Args:
            src_vocab_size (int): Source vocabulary size.
            tgt_vocab_size (int): Target vocabulary size.
            embd_dims (int): Embedding dimensions.
            hidden_size (int): Size/Dimensions for the hidden states.
        """

        super(RNNEncoderDecoderLM, self).__init__()

        # Dummy parameter to track the model device. Do not modify.
        self._dummy_param = torch.nn.Parameter(torch.Tensor(0), requires_grad=False)

        # BEGIN CODE : enc-dec-rnn.init

        # ADD YOUR CODE HERE
        self.src_vocab_size = src_vocab_size
        self.tgt_vocab_size = tgt_vocab_size
        self.embd_dim = embd_dims
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout = dropout

        #Define the encoder and decoder RNNs
        self.RNN_encoder = torch.nn.LSTM(self.embd_dim, self.hidden_size, self.num_layers, batch_first = True, dropout = dropout if num_layers > 1 else 0)
        self.RNN_decoder = torch.nn.LSTM(self.hidden_size, self.hidden_size, self.num_layers, batch_first = True, dropout = dropout if num_layers > 1 else 0)

        # Define a projection layer form hidden layer to target vocab size
        self.linear = torch.nn.Linear(self.hidden_size, self.tgt_vocab_size)

        #Define the embeddings to both the encoder and decoder
        self.embedding_encoder = torch.nn.Embedding(self.src_vocab_size, self.embd_dim)
        self.embedding_decoder = torch.nn.Embedding(self.tgt_vocab_size, self.hidden_size)

        # END CODE

    @property
    def device(self):
        """ Returns the device the model parameters are on. """
        return self._dummy_param.device

    def forward(self, inputs, decoder_inputs, decoder_hidden_state=None):
        """ Performs a forward pass over the encoder-decoder network.

            Accepts inputs for the encoder, inputs for the decoder, and hidden state for
                the decoder to continue generation after the given input.

        Args:
            inputs (torch.Tensor): tensor of shape [batch_size?, max_seq_length]
            decoder_inputs (torch.Tensor): tensor of shape [batch_size?, 1]
            decoder_hidden_state (any): tensor to represent decoder hidden state from time step T-1.

        Returns:
            tuple[torch.Tensor, any]: output from the decoder, and associated hidden state for the next step.
            Decoder outputs should be log probabilities over the target vocabulary.
        """

        # BEGIN CODE : enc-dec-rnn.forward

        # ADD YOUR CODE HERE
        #decoder_inputs would always be a tensor of padded sequence, since we take care of changing to a tensor in the train_step
        embedded_decoder_inputs = self.embedding_decoder(decoder_inputs)# dim (batch, 1, hiden_size)

        #If the inputs is a padded sequence, then we'd have to convert to a packedSequence
        if isinstance(inputs, torch.nn.utils.rnn.PackedSequence):
          inputs = torch.nn.utils.rnn.pad_packed_sequence(inputs, batch_first = True)[0]

        #Embedded the inputs
        embedded_inputs = self.embedding_encoder(inputs) #dim (batch, max_seq_length, embedding)

        pad_token = 0
        seq_lengths = (inputs != pad_token).sum(dim=1).cpu()
        #We assign the embedded_inputs to packedSequence

        embedded_inputs = torch.nn.utils.rnn.pack_padded_sequence(embedded_inputs, seq_lengths, batch_first=True, enforce_sorted=False)


         #If the decoder input is None, then make the encoder input as the decoder input
        if(decoder_hidden_state is None):
          _, encoder_hidden_outputs = self.RNN_encoder(embedded_inputs)
          decoder_hidden_state = encoder_hidden_outputs #dim (num_layers, batch_size, hidden_size)

        #Calcualte the decoder outputs
        decoder_output, decoder_output_hidden_state = self.RNN_decoder(embedded_decoder_inputs, decoder_hidden_state) #decoder_output_hidden_state dim (num_layers, batch, hidden_size)

        #Linearly project the decoder outputs from the hidden layer dimentions to the target vocab size
        output = self.linear(decoder_output)#dim (batch, 1, tgt_vocab_size)

        #Calculate the log-softmax of the output
        log_softmax = torch.nn.functional.log_softmax(output, dim = -1)

        return log_softmax, decoder_output_hidden_state

        # END CODE

    def log_probability(self, seq_x, seq_y):
        """ Compute the conditional log probability of seq_y given seq_x, i.e., log P(seq_y | seq_x).

        Args:
            seq_x (torch.tensor): Input sequence of tokens.
            seq_y (torch.tensor): Output sequence of tokens.

        Returns:
            float: Log probability of seq_y given seq_x
        """

        # BEGIN CODE : enc-dec-rnn.log_probability

        # ADD YOUR CODE HERE
        log_prob = 0

        #Make the seq_x and seq_y into the proper format i.e. dim (batch, sequence_length)
        seq_x = seq_x.unsqueeze(0)# dim (1, seq_length)
        seq_y = seq_y.unsqueeze(0)# dim (1, seq_length)

        #First input to the decoder is going to be the BOS token
        decoder_input = seq_y[:, 0].unsqueeze(0)# dim (1, 1)

        #First forward pass on the encoder-decoder
        output, decoder_output_hidden_state = self.forward(seq_x, decoder_input, None)
        #Output would be of the dim (batch, 1, tgt_vocab_size) and the output hidden state would be #dim (num_layers, 1, hidden_size)

        #The output is the log-softmax of the logits, so we need to take the seq_y[0, 0]th index from the output
        log_prob += output[0, 0, seq_y[0, 1]].sum().item()

        #Iterate through the rest of the sequence length with teacher forcing
        for seq_index in range(1, seq_y.shape[1] - 1):
          decoder_input = seq_y[:, seq_index].unsqueeze(0)# dim (1, 1)

          output, decoder_output_hidden_state = self.forward(seq_x, decoder_input, decoder_output_hidden_state)
          #Output would be of the dim (batch, 1, tgt_vocab_size) and the output hidden state would be #dim (num_layers, 1, hidden_size)

          #The output is the log-softmax of the logits, so we need to take the seq_y[0, seq_index]th index from the output
          log_prob += output[0, 0, seq_y[0, seq_index + 1]].sum().item()
        return log_prob

        # END CODE

## ==== END EVALUATION PORTION

To train the above model, implement for training and evaluation steps in the `RNNEncoderDecoderTrainer` class below:

In [24]:
## ==== BEGIN EVALUATION PORTION

class RNNEncoderDecoderTrainer(Trainer):
    """ Performs model training for RNN-based Encoder-Decoder models. """

    def __init__(self, directory, model, criterion, optimizer):
        """ Initializes the trainer.

        Args:
            directory (str): Directory to save checkpoints and the model data in.
            model (torch.nn.Module): Torch model to train.
            criterion (torch.nn.Function): Loss Criterion.
            optimizer (torch.optim.Optimizer): Optimizer to use.
        """

        super(RNNEncoderDecoderTrainer, self).__init__(directory, model, criterion, optimizer)

    @staticmethod
    def make_dataloader(dataset, shuffle_data=True, batch_size=8, collate_fn=None):
        """ Create a dataloader for a torch Dataset.

        Args:
            dataset (torch.utils.data.Dataset): Dataset to process.
            shuffle_data (bool, optional): If true, shuffles the data. Defaults to True.
            batch_size (int, optional): Number of items per batch. Defaults to 8.
            collate_fn (function, optional): Function to collate instances in a batch.

        Returns:
            torch.utils.data.DataLoader: Dataloader over the given data, post processing.
        """

        # BEGIN CODE : rnn-enc-dec-trainer.make_dataloader

        # ADD YOUR CODE HERE
        return torch.utils.data.DataLoader(
            dataset,
            batch_size = batch_size,
            shuffle = shuffle_data,
            collate_fn = collate_fn
        )

        # END CODE

    def train_step(self, x_batch, y_batch):
        """ Performs a step of training, on the training batch.

        Args:
            x_batch (torch.Tensor): Input batch tensor, of shape [batch_size, *instance_shape].
              For RNNs this is [batch_size, src_padding] or a torch.nn.utils.rnn.PackedSequence of varying lengths per batch (depends on padding).
            y_batch (torch.Tensor): Output batch tensor, of shape [batch_size, *instance_shape].
              For RNNs this is [batch_size, tgt_padding] or a torch.nn.utils.rnn.PackedSequence of varying lengths per batch (depends on padding).

        Returns:
            float: Training loss with the current model, on this batch.
        """

        # BEGIN CODE : rnn-enc-dec-trainer.train_step

        # ADD YOUR CODE HERE
        self.model.train()
        self.optimizer.zero_grad()

        #Move the batches to the device
        x_batch = x_batch.to(self.model.device)# dim (batch, src_vocab_size)
        y_batch = y_batch.to(self.model.device)# dim (batch, tgt_vocab_size)

        #If the y_batch is a packedSequence, then make it a padded tensor
        if isinstance(y_batch, torch.nn.utils.rnn.PackedSequence):
          y_batch = torch.nn.utils.rnn.pad_packed_sequence(y_batch, batch_first = True)[0]

        #Determine the max sequence length
        max_seq_length = y_batch.shape[1]
        # Make a forward pass
        decoder_input = y_batch[:, 0].unsqueeze(1)# dim (batch, 1)

        output, decoder_output_hidden_state = self.model(x_batch, decoder_input, None)
        #Output would be of the dim (batch, 1, tgt_vocab_size) and the output hidden state would be #dim (num_layers, 1, hidden_size)

        #Calculate the loss
        loss = self.criterion(output.squeeze(1), y_batch[:, 1])

        #Calculate the losses on next outputs
        for seq_index in range(1, y_batch.shape[1] -1):
          decoder_input = y_batch[:, seq_index].unsqueeze(1) # dim (batch, 1)
          output, decoder_output_hidden_state = self.model(x_batch, decoder_input, decoder_output_hidden_state)
          #Output: dim (batch, 1, tgt_vocab_size), decoder_ouput_hidden_state: dim (num_layers, batch, hidden_size)

          #Output already has the log-softmax operation so, unsqueeze the output and add the loss
          loss += self.criterion(output.squeeze(1), y_batch[:, seq_index + 1])

        #back-prop on the loss
        loss = loss / (max_seq_length - 1)
        loss.backward()

        #Update the weights
        self.optimizer.step()

        #Return the Loss
        return loss.item()
        # END CODE

    def eval_step(self, validation_dataloader):
        """ Perfoms an evaluation step, on the validation dataloader.

        Args:
            validation_dataloader (torch.utils.data.DataLoader): Dataloader for the validation dataset.

        Returns:
            float: Validation loss with the current model checkpoint.
        """

        # BEGIN CODE : rnn-enc-dec-trainer.eval_step

        # ADD YOUR CODE HERE
        self.model.eval()
        val_loss_list = []
        with torch.no_grad():
          for val_x, val_y in validation_dataloader:
            #Move the tensors to device
            val_x = val_x.to(self.model.device)
            val_y = val_y.to(self.model.device)
            #If the y_batch is a packedSequence, then make it a padded sequence
            if isinstance(val_y, torch.nn.utils.rnn.PackedSequence):
              val_y = torch.nn.utils.rnn.pad_packed_sequence(val_y, batch_first = True)[0]

            #Determine the max sequence length
            max_seq_length = val_y.shape[1]

            #Calcualte the outputs
            decoder_input = val_y[:, 0].unsqueeze(1) # dim(batch, 1)
            output, decoder_output_hidden_state = self.model(val_x, decoder_input, None)
            #Output would be of the dim (batch, 1, tgt_vocab_size) and the output hidden state would be #dim (num_layers, 1, hidden_size)
            #Calculate the loss
            loss = self.criterion(output.squeeze(1), val_y[:,1]).sum().item() # Compute the loss for the log-softmax output and the first text output for the entire batch

            #Calcualte the subsequent losses
            for seq_index in range(1, val_y.shape[1] - 1):
              decoder_input = val_y[:, seq_index].unsqueeze(1) #dim(batch, 1)
              output, decoder_output_hidden_state = self.model(val_x, decoder_input, decoder_output_hidden_state)

              #Calculate the loss
              loss += self.criterion(output.squeeze(1), val_y[:,seq_index + 1]).sum().item() # Compute the loss for the log-softmax output and the seq_index chat output for the entire batch
            loss = loss / (max_seq_length - 1)
            val_loss_list.append(loss)
        average_val_loss = sum(val_loss_list) / len(val_loss_list)
        return average_val_loss
        # END CODE

## ==== END EVALUATION PORTION

In [25]:
## == BEGIN EVALUATION PORTION

# Edit the hyperparameters below to your desired values.

# BEGIN CODE : rnn-enc-dec.params

# Add parameters related to the model here.
rnn_enc_dec_params = {
    'src_vocab_size': len(src_tokenizer.inv_vocab.items()),
    'tgt_vocab_size': len(tgt_tokenizer.inv_vocab.items()),
    'embd_dims'     : 100,
    'hidden_size'   : 200,
    'dropout'       : 0.1,
    'num_layers'    : 2
}


# Add parameters related to the dataset processing here.
rnn_enc_dec_data_params = dict(
    src_padding=None,
    tgt_padding=20,
)

# Add parameters related to training here.
rnn_enc_dec_training_params = dict(
    num_epochs=20,
    batch_size=32,
    shuffle=True
    # save_steps=None,
    # eval_steps=None
)

# END CODE

# Do not forget to set a deterministic seed.
torch.manual_seed(42)

model = RNNEncoderDecoderLM(**rnn_enc_dec_params)

# BEGIN CODE : rnn-enc-dec.train

# ADD YOUR CODE HERE
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3, weight_decay = 1e-4)
criterion = torch.nn.NLLLoss()

# END CODE

trainer = RNNEncoderDecoderTrainer(
    os.path.join(DIRECTORY_NAME, "rnn.enc-dec"),
    model, criterion, optimizer
)

## == END EVALUATION PORTION

In [26]:
# Please do not change anything in the following cell.

train_dataset      = TokenizerDataset(train_data     , src_tokenizer, tgt_tokenizer, **rnn_enc_dec_data_params)
validation_dataset = TokenizerDataset(validation_data, src_tokenizer, tgt_tokenizer, **rnn_enc_dec_data_params)

rnn_enc_dec_train_data = dict(
    train_dataset=train_dataset,
    validation_dataset=validation_dataset,
    collate_fn=train_dataset.collate
)

# Resume training from the last checkpoint, if interrupted midway, else begins training from scratch.
trainer.resume()

# Train as per specified training parameters.
trainer.train(**rnn_enc_dec_train_data, **rnn_enc_dec_training_params)

  0%|          | 0/141 [00:00<?, ?it/s]

[>] epoch #1, batch #100: loss: 0.95249248 | val_loss: 0.90695694


In [27]:
# Please do not change anything in the following cell.

# Save the final model, with additional metadata.
trainer.save(metadata={
    'model'   : rnn_enc_dec_params,
    'data'    : rnn_enc_dec_data_params,
    'training': rnn_enc_dec_training_params
})

To validate training, look at sample translations for different examples, and probabilities assigned to different outputs.

Extensive evaluation and comparison against other approaches will be carried out later.

In [28]:
def rnn_greedy_generate(model, seq_x, src_tokenizer, tgt_tokenizer, max_length):
    """ Given a source string, translate it to the target language using the trained model.
        This function should perform greedy sampling to generate the results.

    Args:
        model (nn.Module): RNN Type Encoder-Decoder Model
        seq_x (str): Input string to translate.
        src_tokenizer (Tokenizer): Source language tokenizer.
        tgt_tokenizer (Tokenizer): Target language tokenizer.
        max_length (int): Maximum length of the target sequence to decode.

    Returns:
        str: Generated string for the given input in the target language.
    """

    # BEGIN CODE : enc-dec-rnn.greedy_generate

    # ADD YOUR CODE HERE
    model.eval()
    device = model.device

    # Tokenize input and convert to tensor
    src_tokens = src_tokenizer.encode(seq_x)  # Convert input string to token IDs
    src_tensor = torch.tensor(src_tokens, dtype=torch.long).unsqueeze(0).to(device)  # Shape: (1, seq_len)

    # Prepare the initial decoder input (start token)
    start_token = tgt_tokenizer.special_tokens['[BOS]']
    decoder_input = torch.tensor([[start_token]], dtype=torch.long).to(device)  # Shape: (1, 1)

    decoded_tokens = []

    decoder_hidden = None  # Initialize decoder with None

    if max_length is None:
      max_length = max([len(train_data.Translation[i]) for i in range(len(train_data.Translation))])

    for _ in range(max_length):
        #Generate the log_probs and the decoder_hidden from model
        output_logits, decoder_hidden = model(src_tensor, decoder_input, decoder_hidden)

        # Get the token with the highest probability (greedy decoding)
        next_token = output_logits.argmax(dim=-1)  # Shape: (1, 1)
        next_token_id = next_token.item()

        # Stop decoding if `<end>` token is generated
        if next_token_id == tgt_tokenizer.special_tokens['[EOS]']:
            break

        decoded_tokens.append(next_token_id)
        # Update decoder input for next iteration
        decoder_input = next_token  # Keep shape as (1, 1)

    # Convert token IDs back to text
    generated_text = tgt_tokenizer.decode(decoded_tokens)
    return generated_text

    # END CODE

In [29]:
# Please do not change anything in the following cell.

for _, row in train_data.sample(n=5, random_state=42).iterrows():
    y_pred = rnn_greedy_generate(
        model, row['Name'], src_tokenizer, tgt_tokenizer,
        max_length = rnn_enc_dec_data_params['tgt_padding']
    )

    print("Name                      :", row['Name'])
    print("Translation (Expected)    :", row['Translation'])
    print("Translation (Model)       :", y_pred)

    print()

Name                      : वरिषा
Translation (Expected)    : varisha
Translation (Model)       : ash

Name                      : उजामा
Translation (Expected)    : ujama
Translation (Model)       : ash

Name                      : बिपिन
Translation (Expected)    : bipin
Translation (Model)       : ash

Name                      : मित्ठू
Translation (Expected)    : mitthu
Translation (Model)       : as

Name                      : रामैया
Translation (Expected)    : ramaiya
Translation (Model)       : ash



In [30]:
# Please do not change anything in the following cell.

for _, row in validation_data.sample(n=5, random_state=42).iterrows():
    y_pred = rnn_greedy_generate(
        model, row['Name'], src_tokenizer, tgt_tokenizer,
        max_length = rnn_enc_dec_data_params['tgt_padding']
    )

    print("Name                      :", row['Name'])
    print("Translation (Expected)    :", row['Translation'])
    print("Translation (Model)       :", y_pred)

    print()

Name                      : दीन
Translation (Expected)    : deen
Translation (Model)       : a

Name                      : मुर्शिदा
Translation (Expected)    : murshida
Translation (Model)       : as

Name                      : शबरा
Translation (Expected)    : shabra
Translation (Model)       : a

Name                      : श्रीकांत
Translation (Expected)    : shrikant
Translation (Model)       : as

Name                      : कौशल
Translation (Expected)    : kaushal
Translation (Model)       : ash



In [31]:
# Please do not change anything in the following cell.

output_data = []
for _, row in validation_data.iterrows():
    y_pred = rnn_greedy_generate(
        model, row['Name'], src_tokenizer, tgt_tokenizer,
        max_length = rnn_enc_dec_data_params['tgt_padding']
    )
    output_data.append({ 'Name': row['Name'], 'Translation': y_pred })

pd.DataFrame.from_records(output_data).to_csv(
    os.path.join(DIRECTORY_NAME, "rnn.enc-dec", "outputs.csv"), index=False
)

In [32]:
# Please do not change anything in the following cell.

# Release resources
if 'trainer' in globals():
    del trainer

if 'model' in globals():
    del model

sync_vram()

## Seq-2-Seq Modeling with RNN + Attention

In this module, you'll augment the Encoder-Decoder architecture to utilize attention, by implementing an Attention module that attends over the representations / inputs from the encoder.

Many approaches have been proposed in literature towards implementing attention. You are free to explore and use any implementation of your choice.

Some popular approaches are desribed in the original [paper by Bahdanau et al., 2014 on NMT](https://arxiv.org/abs/1409.0473) and an [exploratory paper by Luong et al, 2015](https://arxiv.org/abs/1508.04025) which explores different effective approaches to attention, including global and local attention.

In [33]:
## ==== BEGIN EVALUATION PORTION

class AttentionModule(torch.nn.Module):
    """ Implements an attention module """

    # Feel free to add additional parameters to __init__
    def __init__(self, input_size):
        """ Initializes the attention module.
            Feel free to declare any parameters as required. """

        super(AttentionModule, self).__init__()

        # BEGIN CODE : attn.init

        # ADD YOUR CODE HERE
        self.W_a = torch.nn.Linear(input_size, input_size)
        self.U_a = torch.nn.Linear(input_size, input_size)
        self.tanh = torch.nn.Tanh()
        self.v_a = torch.nn.Linear (input_size, 1)


        # END CODE

    def forward(self, encoder_outputs, decoder_hidden_state):
        """ Performs a forward pass over the module, computing attention scores for inputs.

        Args:
            encoder_outputs (torch.Tensor): Output representations from the encoder, of shape [batch_size?, src_seq_len, output_dim].
            decoder_hidden_state (torch.Tensor): Hidden state from the decoder at current time step, of appropriate shape as per RNN unit (with optional batch dim).

        Returns:
            torch.Tensor: Attentions scores for given inputs, of shape [batch_size?, 1, src_seq_len]
        """

        # BEGIN CODE : attn.forward

        # ADD YOUR CODE HERE
        #Ensure that decoder_hidden_state dim (batch, 1, dim)
        if len(decoder_hidden_state.shape) == 2:# if shape is dim (batch, dim)
          decoder_hidden_state = decoder_hidden_state.unsqueeze(1)# dim (batch, 1, dim)

        elif len(decoder_hidden_state.shape) == 1:# if shape is dim (dim)
          decoder_hidden_state = decoder_hidden_state.unsqueeze(0).unsqueeze(0)#dim (1, 1, dim)

        #Ensure that encoder_outputs dim (batch, seq_len, dim)
        if len(encoder_outputs.shape) == 2:# if shape is dim (seq_len, dim)
          encoder_outputs = encoder_outputs.unsqueeze(0)# dim (1, seq_len, dim)

        elif len(encoder_outputs.shape) == 1:# if shape is dim (dim)
          encoder_outputs = encoder_outputs.unsqueeze(0).unsqueeze(0)#dim (1, 1 )

        tanh_output = self.tanh(self.W_a(encoder_outputs) + self.U_a(decoder_hidden_state))
        attentions = self.v_a(tanh_output).transpose(1,2)
        softmax_attentions = torch.nn.functional.softmax(attentions, dim=-1)
        return softmax_attentions
        # END CODE

## ==== END EVALUATION PORTION

In [34]:
## ==== BEGIN EVALUATION PORTION

class RNNEncoderDecoderLMWithAttention(torch.nn.Module):
    """ Implements an Encoder-Decoder network, using RNN units, augmented with attention. """

    # Feel free to add additional parameters to __init__
    def __init__(self,src_vocab_size, tgt_vocab_size, embd_dims, hidden_size, num_layers=1, dropout=0.1):
        """ Initializes the encoder-decoder network, implemented via RNNs.

        Args:
            src_vocab_size (int): Source vocabulary size.
            tgt_vocab_size (int): Target vocabulary size.
            embd_dims (int): Embedding dimensions.
            hidden_size (int): Size/Dimensions for the hidden states.
        """

        super(RNNEncoderDecoderLMWithAttention, self).__init__()

        # Dummy parameter to track the model device. Do not modify.
        self._dummy_param = torch.nn.Parameter(torch.Tensor(0), requires_grad=False)

        # BEGIN CODE : enc-dec-rnn-attn.init

        # ADD YOUR CODE HERE
        self.src_vocab_size = src_vocab_size
        self.tgt_vocab_size = tgt_vocab_size
        self.embd_dim = embd_dims
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout = dropout

        #Define the encoder and decoder RNNs
        self.RNN_encoder = torch.nn.LSTM(self.embd_dim, self.hidden_size, self.num_layers, batch_first = True, dropout = dropout if num_layers > 1 else 0)
        self.RNN_decoder = torch.nn.LSTM(self.hidden_size, self.hidden_size, self.num_layers, batch_first = True, dropout = dropout if num_layers > 1 else 0)

        # Define a projection layer form hidden layer to target vocab size
        self.linear = torch.nn.Linear(self.hidden_size, self.tgt_vocab_size)

        #Define the embeddings to both the encoder and decoder
        self.embedding_encoder = torch.nn.Embedding(self.src_vocab_size, self.embd_dim)
        self.embedding_decoder = torch.nn.Embedding(self.tgt_vocab_size, self.hidden_size)

        #Define Attention
        self.attention = AttentionModule(self.hidden_size)


        # END CODE

    @property
    def device(self):
        return self._dummy_param.device

    def log_probability(self, seq_x, seq_y):
        """ Compute the conditional log probability of seq_y given seq_x, i.e., log P(seq_y | seq_x).

        Args:
            seq_x (torch.tensor): Input sequence of tokens, of shape [src_seq_len] (no batch dim)
            seq_y (torch.tensor): Output sequence of tokens, of shape [tgt_seq_len] (no batch dim)

        Returns:
            float: Log probability of generating sequence y, given sequence x.
        """

        # BEGIN CODE : enc-dec-rnn-attn.probability

        # ADD YOUR CODE HERE
        #TODO: Check this function
        log_prob = 0

        #Make the seq_x and seq_y into the proper format i.e. dim (batch, sequence_length)
        seq_x = seq_x.unsqueeze(0)# dim (1, seq_length)
        seq_y = seq_y.unsqueeze(0)# dim (1, seq_length)

        #First input to the decoder is going to be the BOS token
        decoder_input = seq_y[:, 0].unsqueeze(0)# dim (1, 1)

        #First forward pass on the encoder-decoder
        output, decoder_output_hidden_state = self.forward(seq_x, decoder_input, None)
        #Output would be of the dim (batch, 1, tgt_vocab_size) and the output hidden state would be #dim (num_layers, 1, hidden_size)

        #The output is the log-softmax of the logits, so we need to take the seq_y[0, 0]th index from the output
        log_prob += output[0, 0, seq_y[0, 1]].sum().item()

        #Iterate through the rest of the sequence length with teacher forcing
        for seq_index in range(1, seq_y.shape[1] - 1):
          decoder_input = seq_y[:, seq_index].unsqueeze(0)# dim (1, 1)

          output, decoder_output_hidden_state = self.forward(seq_x, decoder_input, decoder_output_hidden_state)
          #Output would be of the dim (batch, 1, tgt_vocab_size) and the output hidden state would be #dim (num_layers, 1, hidden_size)

          #The output is the log-softmax of the logits, so we need to take the seq_y[0, seq_index]th index from the output
          log_prob += output[0, 0, seq_y[0, seq_index + 1]].sum().item()
        return log_prob

        # END CODE

    def attentions(self, seq_x, terminate_token, max_length):
        """ Obtain attention over a sequence for decoding to the target language.

        Args:
            seq_x (torch.tensor): Tensor representing the source sequence, of shape [src_seq_len] (no batch dim)
            terminate_token (int): Token to use as EOS, to stop generating outputs.
            max_length (int): Maximum length to use to terminate the sampling.

        Returns:
            tuple[torch.tensor, torch.tensor]:
                A tuple of two tensors: the attentions over individual output tokens ([tgt_seq_len, src_seq_len])
                and the best output tokens ([tgt_seq_len]) per sequence step, based on greedy sampling.
        """

        # BEGIN CODE : rnn-enc-dec-attn.attentions

        # ADD YOUR CODE HERE
        self.eval()
        inputs = seq_x.unsqueeze(0).to(self.device)# dim(1, src_seq_len)
        decoder_inputs = None
        decoder_hidden_state = None
        output_attention = True
        attention_list = []
        decoder_outputs = []

        count = 0
        while count < max_length:
          log_probs, decoder_hidden_state, attentions = self.forward(inputs, decoder_inputs, decoder_hidden_state, output_attention)
          #log_probs dim (batch_size, 1, tgt_vocab_size), decoder_hidden_state, attentions :  dim (1, 1, src_seq_len)
          # Get the token with the highest probability (greedy decoding)
          next_token = log_probs.argmax(dim=-1)  # Shape: (1, 1)
          next_token_id = next_token.item()

          # Stop decoding if `<end>` token is generated
          if next_token_id == terminate_token:
              break

          decoder_outputs.append(next_token_id)
          attention_list.append(attentions.squeeze(1))
          # Update decoder input for next iteration
          decoder_inputs = next_token  # Keep shape
          count += 1


        if attention_list:
          attention_list = torch.stack(attention_list, dim=0).squeeze(1)
        else:
          attention_list = torch.empty(0, inputs.shape[1])  # Handle edge cases where no attention was stored

        decoder_outputs = torch.tensor(decoder_outputs, dtype=torch.long)

        return attention_list.cpu(), decoder_outputs.cpu()

        # END CODE

    def forward(self, inputs, decoder_inputs=None, decoder_hidden_state=None, output_attention=False):
        """ Performs a forward pass over the encoder-decoder network.

            Accepts inputs for the encoder, inputs for the decoder, and hidden state for
                the decoder to continue generation after the given input.

        Args:
            inputs (torch.Tensor): tensor of shape [batch_size?, src_seq_len]
            decoder_inputs (torch.Tensor): Decoder inputs, as tensor of shape [batch_size?, 1]
            decoder_hidden_state (any): tensor to represent decoder hidden state from time step T-1.
            output_attention (bool): If true, this function should also return the
                associated attention weights for the time step, of shape [batch_size?, 1, src_seq_len].

        Returns:
            tuple[torch.Tensor, any]: output from the decoder, and associated hidden state for the next step.

            Decoder outputs should be log probabilities over the target vocabulary.

        Example:
        >>> model = RNNEncoderDecoderWithAttention(*args, **kwargs)
        >>> output, hidden = model(..., output_attention=False)
        >>> output, hidden, attn_weights = model(..., output_attention=True)
        """

        # BEGIN CODE : enc-dec-rnn-attn.forward

        # ADD YOUR CODE HERE

        #Decoder_inputs would always be a tensor of padded sequence, since we take care of changing to a tensor in the train_step

        #If the inputs is a padded sequence, then we'd have to convert to a packedSequence
        if isinstance(inputs, torch.nn.utils.rnn.PackedSequence):
          inputs = torch.nn.utils.rnn.pad_packed_sequence(inputs, batch_first = True)[0]

        #Embedded the inputs token
        embedded_inputs = self.embedding_encoder(inputs) # dim (1, src_seq_len, embd_dim)

        pad_token = 0
        seq_lengths = (inputs != pad_token).sum(dim=1).cpu()
        #We assign the embedded_inputs to packedSequence

        embedded_inputs = torch.nn.utils.rnn.pack_padded_sequence(embedded_inputs, seq_lengths, batch_first=True, enforce_sorted=False)

        #Generate the encoder output and the encoder hidden state
        encoder_outputs, encoder_hidden_state = self.RNN_encoder(embedded_inputs, decoder_hidden_state) #
        #encoder_outputs would be packedSequence since we send packed sequence
        #encoder_hidden_state : tupple(h_t: (num_layer, batch_size, hidden_size), c_t: (num_layer, batch_size, hidden_size))

        # If the decoder_hidden_state is None, then make it encoder_hidden_state
        if decoder_hidden_state is None:
          decoder_hidden_state = encoder_hidden_state # dim (1, batch_size, hidden_size)

        # If the decoder inputs is None, then make the decoder_inputs is BOS token for the entire batch, so dim (batch_size, 1)
        if decoder_inputs is None:
          start_token = tgt_tokenizer.special_tokens['[BOS]'] #dim (1,1)
          decoder_inputs = torch.tensor([[start_token]] * inputs.shape[0], dtype=torch.long).to(self.device) # dim (batch_size, 1)

        #decoder_inputs is dim (batch_size, 1)
        decoder_input = self.embedding_decoder(decoder_inputs) # dim (batch_size, 1, hidden_size)

        #generate the attention scores
        encoder_outputs = torch.nn.utils.rnn.pad_packed_sequence(encoder_outputs, batch_first = True)[0]
        #encoder_outputs would be dim (batch_size, src_seq_len, hidden_size)

        # Context Calculation
        # Let's first calculate the context vector for h_t (num_layer, batch_size, hidden_size)

        # Inititlaize c_h_t with None
        c_h_t = None

        #Now iterate over the num_layers to calculate the context vector taking the h_t from every layer
        num_layers = self.num_layers
        h_t = decoder_hidden_state[0]
        for i in range(num_layers):
          decoder_hidden_state_attn = h_t[i, :, :].unsqueeze(0) # dim (1, batch_size, hidden_size)
          attentions_output = self.attention(encoder_outputs, decoder_hidden_state_attn.transpose(0,1)) # dim (batch_size, 1, src_seq_len)
          C = torch.bmm(attentions_output, encoder_outputs) # dim (batch_size, 1, hidden_size)
          # Concat the C with c_h_t on the num_layers dimentions
          if c_h_t is None:
            c_h_t = C.transpose(0,1) # dim (1, batch_size, hidden_size)
          else:
            c_h_t = torch.cat((c_h_t, C.transpose(0,1)), dim = 0) # dim (num_layers, batch_size, hidden_size)

        # Now calcualte the context vector for c_t (num_layer, batch_size, hidden_size)

        #Initialize the c_c_t with None
        c_c_t = None

        #Iterate over the num_layers to calculate the context vector taking the c_t from every layer
        c_t = decoder_hidden_state[1]
        for i in range(num_layers):
          decoder_hidden_state_attn = c_t[i, :, :].unsqueeze(0) # dim (1, batch_size, hidden_size)
          attentions = self.attention(encoder_outputs, decoder_hidden_state_attn.transpose(0,1)) # dim (batch_size, 1, src_seq_len)
          C = torch.bmm(attentions, encoder_outputs)
          # Concat the C with the c_c_t on the num_layers dimentions
          if c_c_t is None:
            c_c_t = C.transpose(0,1) # dim (1, batch_size, hidden_size)
          else:
            c_c_t = torch.cat((c_c_t, C.transpose(0,1)), dim = 0) # dim (num_layers, batch_size, hidden_size)

        #decoder hidden state woudl be the tupple (c_h_t, c_c_t)
        decoder_hidden_state = (c_h_t, c_c_t)

        #pass through the RNN_decoder
        decoder_output, decoder_hidden_state = self.RNN_decoder(decoder_input, decoder_hidden_state)
        # decoder_output dim (batch_size, 1, hidden_size), decoder_hidden_state : tupple(c_h_t: (num_layer, batch_size, hidden_size), c_c_t: (num_layer, batch_size, hidden_size))

        #linear project to tgt_vocab_size
        decoder_output = self.linear(decoder_output) # dim (batch_size, 1, tgt_vocab_size)

        #apply log_softmax
        decoder_output = torch.nn.functional.log_softmax(decoder_output, dim=-1) # dim (batch_size, 1, tgt_vocab_size)

        if output_attention:
          return decoder_output, decoder_hidden_state, attentions_output
        else:
          return decoder_output, decoder_hidden_state
        # END CODE

## ==== END EVALUATION PORTION

In [36]:
## == BEGIN EVALUATION PORTION

# Edit the hyperparameters below to your desired values.

# BEGIN CODE : rnn-enc-dec-attn.params

# Add parameters related to the model here.
rnn_enc_dec_attn_params = {
    'src_vocab_size': len(src_tokenizer.inv_vocab.items()),
    'tgt_vocab_size': len(tgt_tokenizer.inv_vocab.items()),
    'embd_dims'     : 100,
    'hidden_size'   : 200,
    'dropout'       : 0.1,
    'num_layers'    : 2
}

# Add parameters related to the dataset processing here.
rnn_enc_dec_attn_data_params = dict(
    src_padding=None,
    tgt_padding=20,
)

# Add parameters related to training here.
rnn_enc_dec_attn_training_params = dict(
    num_epochs=20,
    batch_size=32,
    shuffle=True
    # save_steps=None,
    # eval_steps=None
)

# END CODE

# Do not forget to set a deterministic seed.
torch.manual_seed(42)

model = RNNEncoderDecoderLMWithAttention(**rnn_enc_dec_attn_params)

# BEGIN CODE : rnn-enc-dec-attn.train

# ADD YOUR CODE HERE
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3, weight_decay = 1e-4)
criterion = torch.nn.NLLLoss()

# END CODE

trainer = RNNEncoderDecoderTrainer(
    os.path.join(DIRECTORY_NAME, "rnn.enc-dec.attn"),
    model, criterion, optimizer
)

## == END EVALUATION PORTION

In [37]:
# Please do not change anything in the following cell.

train_dataset      = TokenizerDataset(train_data     , src_tokenizer, tgt_tokenizer, **rnn_enc_dec_attn_data_params)
validation_dataset = TokenizerDataset(validation_data, src_tokenizer, tgt_tokenizer, **rnn_enc_dec_attn_data_params)

rnn_enc_dec_attn_train_data = dict(
    train_dataset=train_dataset,
    validation_dataset=validation_dataset,
    collate_fn=train_dataset.collate
)

# Resume training from the last checkpoint, if interrupted midway, otherwise starts from scratch.
trainer.resume()

# Train as per specified training parameters.
trainer.train(**rnn_enc_dec_attn_train_data, **rnn_enc_dec_attn_training_params)

  0%|          | 0/141 [00:00<?, ?it/s]

[>] epoch #1, batch #100: loss: 1.00696337 | val_loss: 0.95711764


In [ ]:
# Please do not change anything in the following cell.

# Save the final model, with additional metadata.
trainer.save(metadata={
    'model'   : rnn_enc_dec_attn_params,
    'data'    : rnn_enc_dec_attn_data_params,
    'training': rnn_enc_dec_attn_training_params
})

We can validate the model using a few simple tests as below:

In [ ]:
# Please do not change anything in the following cell.

for _, row in train_data.sample(n=5, random_state=42).iterrows():
    y_pred = rnn_greedy_generate(
        model, row['Name'], src_tokenizer, tgt_tokenizer,
        max_length = rnn_enc_dec_attn_data_params['tgt_padding']
    )

    print("Name                      :", row['Name'])
    print("Translation (Expected)    :", row['Translation'])
    print("Translation (Model)       :", y_pred)

    print()

In [ ]:
# Please do not change anything in the following cell.

for _, row in validation_data.sample(n=5, random_state=42).iterrows():
    y_pred = rnn_greedy_generate(
        model, row['Name'], src_tokenizer, tgt_tokenizer,
        max_length = rnn_enc_dec_attn_data_params['tgt_padding']
    )

    print("Name                      :", row['Name'])
    print("Translation (Expected)    :", row['Translation'])
    print("Translation (Model)       :", y_pred)

    print()

It may also be useful to look at attention maps for different examples:

In [ ]:
# Please do not change anything in the following cell.

def visualize_attention(src_glyphs, tgt_glyphs, attention, axes):
    axes.matshow(attention.numpy(), cmap='bone')

    axes.set_xticks(numpy.arange(len(src_glyphs)), labels=src_glyphs)
    axes.set_yticks(numpy.arange(len(tgt_glyphs)), labels=tgt_glyphs)

In [ ]:
# Please do not change anything in the following cell.

pyplot.figure(figsize=(12, 10))

src_id_to_token = inverse_vocabulary(src_tokenizer)
tgt_id_to_token = inverse_vocabulary(tgt_tokenizer)

with torch.no_grad():
    for i, row in train_data.sample(n=4, random_state=42, ignore_index=True).iterrows():
        src_tokens = torch.tensor(src_tokenizer.encode(row['Name']))
        attentions, tgt_tokens = model.attentions(src_tokens, tgt_tokenizer.get_special_tokens()['[EOS]'], max_length=50)
        src_glyphs = apply_inverse_vocab(src_tokens.tolist(), src_id_to_token)
        tgt_glyphs = apply_inverse_vocab(tgt_tokens.tolist(), tgt_id_to_token)
        axes = pyplot.subplot(2, 2, i+1)
        visualize_attention(src_glyphs, tgt_glyphs, attentions, axes)

In [ ]:
# Please do not change anything in the following cell.

output_data = []
for _, row in validation_data.iterrows():
    y_pred = rnn_greedy_generate(
        model, row['Name'], src_tokenizer, tgt_tokenizer,
        max_length = rnn_enc_dec_attn_data_params['tgt_padding']
    )
    output_data.append({ 'Name': row['Name'], 'Translation': y_pred })

pd.DataFrame.from_records(output_data).to_csv(
    os.path.join(DIRECTORY_NAME, "rnn.enc-dec.attn", "outputs.csv"), index=False
)

In [ ]:
# Please do not change anything in the following cell.

# Release resources
if 'trainer' in globals():
    del trainer

if 'model' in globals():
    del model

sync_vram()

## Evaluation

In the last few modules, you have implemented different approaches towards transliteration of Indian names to English. To assess how well different systems perform, it is useful to compute different metrics, which assess different properties:

- **Accuracy**: From a parallel corpus, number of translations the model got exactly right. Higher the better. Note that this makes sense only for this task. and lacks granularity.
- **Edit Distance**: Number of edits at the character level (insertions, deletions, substitutions) required to transform your model's outputs to a reference translation. Lower the better.
- **Character Error Rate (CER)**: The rate at which your system/model makes mistakes at the character level. Lower the better.
- **Token Error Rate (TER)**: The rate at which your system/model makes mistakes at the token level. Lower the better. Depending on your tokenizer implementation, could be the same as CER.
- **BiLingual Evaluation Understudy (BLEU)**: Proposed by [Papineni et al., 2002](https://aclanthology.org/P02-1040/), BLEU is a metric that assess the quality of a translation against reference translations through assessing n-gram overlap. Higher the better.

Since accents and half-letters exist as separate characters in the Unicode specification, and can change the interpretation of the output, metrics that operate at the character level will treat these separately.

In [ ]:
# Please do not change anything in the following cell.

class Evaluator:
    """ Class to handle all the logic concerning the evaluation of trained models.  """

    def __init__(self, src_tokenizer, tgt_tokenizer) -> None:
        """ Initializes the evaluator.

        Args:
            src_tokenizer (Tokenizer): Tokenizer for input strings in the source language.
            tgt_tokenizer (Tokenizer): Tokenizer for output strings in the target language.
        """

        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.src_tokenizer = src_tokenizer
        self.tgt_tokenizer = tgt_tokenizer
        self.decoding_method = None

    def set_decoding_method(self, decoding_method):
        """ Sets the decoding method to use with models.
                The evaluation function will use the set decoding method to generate outputs from the model.

        Args:
            decoding_method (function): Decoding method.
                Must accept the model instance, the input string, and tokenizers as arguments.
                Can accept additional arguments if required.
        """

        self.decoding_method = decoding_method

    @staticmethod
    def decompose(string):
        """ Decomposes a string into a set of tokens.

        Args:
            string (str): String to decompose.

        Returns:
            list[str]: List of characters from the string.
        """
        return unicodedata.normalize('NFKD', string).encode('utf-8')

    @staticmethod
    def levenshtein_distance(string1, string2):
        """ Computes the levensthein distance between two strings.

        Args:
            string1 (list[any]): Sequence A.
            string2 (list[any]): Sequence B.

        Returns:
            tuple[int, int, int]: Number of insertions + deletions, substitutions and no-ops.
        """

        costs = [
            [ 0 for j in range(len(string2)+1) ]
            for i in range(len(string1)+1)
        ]

        # Prepare matrix of costs.
        for i in range(len(string1)+1): costs[i][0] = i
        for j in range(len(string2)+1): costs[0][j] = j
        for i in range(1, len(string1)+1):
            for j in range(1, len(string2)+1):
                costs[i][j] = min(
                    costs[i][j-1] + 1,
                    costs[i-1][j] + 1,
                    costs[i-1][j-1] + (0 if string1[i-1] == string2[j-1] else 1)
                )

        # Decode matrix in backward manner for actual operation counts.
        c_ins_del, c_sub, c_noop = 0, 0, 0

        i, j = len(string1), len(string2)
        while i > 0 or j > 0:
            if i > 0 and costs[i][j] == costs[i-1][j] + 1:
                c_ins_del += 1
                i -= 1
            elif j > 0 and costs[i][j] == costs[i][j-1] + 1:
                c_ins_del += 1
                j -= 1
            elif i > 0 and j > 0:
                if string1[i-1] == string2[j-1]:
                    c_noop += 1
                else:
                    c_sub += 1
                i, j = i-1, j-1
            else:
                break

        return c_ins_del, c_sub, c_noop

    @staticmethod
    def accuracy(y_true, y_pred):
        """ Computes the accuracy of the predictions, against a reference set of predictions.

        Args:
            y_true (list[str]): Actual translations.
            y_pred (list[str]): Generated translations.

        Returns:
            float: Accuracy score, between 0 and 1.
        """
        return sum(yi_true == yi_pred for yi_true, yi_pred in zip(y_true, y_pred)) / len(y_pred)

    @classmethod
    def char_error_rate(cls, y_true, y_pred):
        """ Computes the character level error rate (CER) of the set of
            predictions against the reference translations.

        Args:
            y_true (list[str]): Actual translations.
            y_pred (list[str]): Generated translations.

        Returns:
            float: CER score, between 0 and 1. Lower the better.
        """

        cer_score = 0

        for yi_true, yi_pred in zip(y_true, y_pred):
            yi_true, yi_pred = cls.decompose(yi_true), cls.decompose(yi_pred)
            c_ins_del, c_sub, c_noop = cls.levenshtein_distance(yi_true, yi_pred)
            cer_score += (c_ins_del + c_sub) / (c_ins_del + c_sub + c_noop)

        return cer_score / len(y_true)

    def token_error_rate(self, y_true, y_pred):
        """ Computes the token level error rate (TER) of the set of
            predictions against the reference translations.

        Args:
            y_true (list[str]): Actual translations.
            y_pred (list[str]): Generated translations.

        Returns:
            float: TER score, between 0 and 1. Lower the better.
        """

        ter_score = 0

        for yi_true, yi_pred in zip(y_true, y_pred):
            yi_true = self.tgt_tokenizer.encode(yi_true, add_start=False, add_end=False)
            yi_pred = self.tgt_tokenizer.encode(yi_pred, add_start=False, add_end=False)
            t_ins_del, t_sub, t_noop = self.levenshtein_distance(yi_true, yi_pred)
            ter_score += (t_ins_del + t_sub) / (t_ins_del + t_sub + t_noop)

        return ter_score / len(y_true)

    @classmethod
    def bleu_score(cls, y_true, y_pred):
        """ Computes the average BLEU score of the set of predictions against the reference translations.

            Uses default parameters and equal weights for all n-grams, with max N = 4. (Thus computes BLEU-4).
            Uses a smoothing method for the case of missing n-grams.

        Args:
            y_true (list[str]): Actual translations.
            y_pred (list[str]): Generated translations.

        Returns:
            float: BLEU-4 score, the higher the better.
        """

        y_true = [ [ cls.decompose(yi) ] for yi in y_true ]
        y_pred = [ cls.decompose(yi) for yi in y_pred ]

        smoothing = bleu_score.SmoothingFunction()

        return bleu_score.corpus_bleu(
            y_true, y_pred,
            smoothing_function=smoothing.method1
        )

    def evaluate(self, model_path, data, reference_outputs, **decoding_kwargs):
        """ Performs the evaluation of a specified model over given data.

        Args:
            model_path (str): Path to load the model from. Must have a model.pt file.
            data (list[str]): List of input strings to translate.
            reference_outputs (list[str]): List of output strings to use as reference.
            decoding_kwargs (dict[str, any]): Additional arguments to forward to the decoding method.
                This could be for instance, max_length for a greedy decoding method.

        Raises:
            ValueError: If the decoding method is not set apriori.
        """

        if self.decoding_method is None:
            raise ValueError(f"{self.evaluate.__name__}: no decoding method is set, assign before use.")

        # Load the model to the active device.
        model = torch.load(os.path.join(model_path, 'model.pt'), map_location=self.device, weights_only=False)

        # Set model use parameters.
        model.to(self.device)
        model.eval()

        # Generate outputs.
        generated_outputs = []
        with torch.no_grad():
            for seq_x in data:
                generated_outputs.append(self.decoding_method(
                    model, seq_x, self.src_tokenizer,
                    self.tgt_tokenizer, **decoding_kwargs
                ))

        accuracy_score = self.accuracy(reference_outputs, generated_outputs)
        cer_score      = self.char_error_rate(reference_outputs, generated_outputs)
        ter_score      = self.token_error_rate(reference_outputs, generated_outputs)
        blue_score     = self.bleu_score(reference_outputs, generated_outputs)

        print("EVALUATION:", ">", "accuracy:", f"{accuracy_score:.2%}")
        print("EVALUATION:", ">", "CER     :", f"{cer_score:.2%}")
        print("EVALUATION:", ">", "TER     :", f"{ter_score:.2%}")
        print("EVALUATION:", ">", "BLEU    :", f"{blue_score:.4f}")
        print()

        # Free resources once evaluation is complete.
        del model
        sync_vram()

In [ ]:
# Please do not change anything in the following cell.

evaluator = Evaluator(src_tokenizer, tgt_tokenizer)

# Use greedy decoding for producing outputs.
evaluator.set_decoding_method(rnn_greedy_generate)

# Evaluate enc-dec-rnn
print("EVALUATION:", "enc-dec-rnn")
evaluator.evaluate(
    os.path.join(DIRECTORY_NAME, "rnn.enc-dec"),
    validation_data['Name'], validation_data['Translation'],
    max_length = rnn_enc_dec_data_params['tgt_padding']
)

# Evaluate enc-dec-rnn-attn
print("EVALUATION:", "enc-dec-rnn-attn")
evaluator.evaluate(
    os.path.join(DIRECTORY_NAME, "rnn.enc-dec.attn"),
    validation_data['Name'], validation_data['Translation'],
    max_length = rnn_enc_dec_attn_data_params['tgt_padding']
)

## Decoding Strategy

A conditional language model aims to learn $P_\theta(y | x)$, that is, the probability of the target sequence being $y$ when the input sequence is $x$. This is modeled as $P_{\theta}(y | x) = \prod_{i=1}^{|y|} {P_\theta(y_i | x, y_{1:i-1})}$.

For translation, our goal is to find the sequence that maximizes this conditional probability, i.e. $y^* = \arg \max_{y} P_\theta(y | x)$. $y^*$ is then the 'best' translation for the input sequence $x$. However, computing probabilities for all possible $y$ to find the maximizer is intractable. As a result, decoding strategies are employed to produce reasonable approximations of $y^*$.

In the last module, you evaluated your models through different metrics, but the approach for generating outputs from the model was fixed to greedy decoding, where at each time step, the token to be produced is determined by $y_{i,greedy} := \arg \max_{y_i} P(y_i| x, y_{1:i-1})$. While this approach is fast, $P(y_{greedy}|x)$ may be much less than $P(y^*|x)$. Fortunately, better decoding strategies exist to produce better approximations, however at the cost of higher time complexity. One such strategy is:

- **Beam-Search Decoding**: At every time step, retains $k$ candidate token generations, which are decoded individually (each path is referred as a beam) to obtain $k$ successors per beam. For the next time step, the best $k$ candidates are retained such that conditional probability of the sequence generated so far is maximized. Has a complexity of $O(kV|y|)$, where $V$ is the size of the target vocabulary, and $|y|$ is the target sequence length. Using $k=1$ makes it equivalent to greedy decoding. Implementations also employ length penalties to not be biased towards larger target sequences.

In the next cell, you will implement the above explained Beam-Search Decoding decoding strategy.

In [ ]:
## ==== BEGIN EVALUATION PORTION

# Feel free to add additional parameters to rnn_better_generate, such as k for Beam Search Decoding.
def rnn_better_generate(model, seq_x, src_tokenizer, tgt_tokenizer, max_length, k=5, length_penalty_alpha=0.6):
    """ Given a source string, translate it to the target language using the trained model.
        This function should use a better decoding strategy than greedy decoding (see above) to generate the results.

    Args:
        model (nn.Module): RNN Type Encoder-Decoder Model
        seq_x (str): Input string to translate.
        src_tokenizer (Tokenizer): Source language tokenizer.
        tgt_tokenizer (Tokenizer): Target language tokenizer.
        max_length (int): Maximum length of the target sequence to decode.

    Returns:
        str: Generated string for the given input in the target language.
    """

    # BEGIN CODE : enc-dec-rnn.better_generate

    # ADD YOUR CODE HERE
    model.eval()
    device = model.device

    #Tokenize the input and convert to tensor
    input_tokens = src_tokenizer.encode(seq_x)
    input_tokens = torch.tensor(input_tokens, dtype=torch.long, device=device).unsqueeze(0) # dim (1, seq_len)

    #Prepare the initial decoder input, BOS token
    decoder_input = torch.tensor([[tgt_tokenizer.get_special_tokens()['[BOS]']]], dtype=torch.long, device=device) # dim (1, 1)

    #Let's define a tupple list, curr, containing the current set of token sequence, along with their properties
    properties = {
        "hidden": None,
        "log_prob" : 0.0,
        "token_list" : decoder_input
    }

    # Now define the current beam to store all the beams
    curr_beams = [(decoder_input, properties)]
    #Final sequences
    final_sequences = []

    #Run a loop for the max_length
    for i in range(max_length):
      # Define the newbeams to come out of newbeams
      new_beams = []

      # For the each of the beam tokens, pass the tokens to the model, get the output and the hidden state and log probabilites of top k
      for beam in curr_beams:
        tokens = beam[0]
        properties = beam[1]
        if tokens == tgt_tokenizer.get_special_tokens()['[EOS]']:
          token_list = torch.cat((properties["token_list"], tokens))
          score = properties['log_prob']
          normailizer = len(properties['token_list']) ** length_penalty_alpha
          score /= normailizer
          final_sequences.append((token_list, score))
          continue
        decoder_output, decoder_hidden_states = model(input_tokens, tokens, properties['hidden'])

        #Take the top K probablities
        top_k_probabilities, top_k_indices = torch.topk(decoder_output, k, dim = -1)
        for new_prob, new_token in zip(top_k_probabilities.squeeze(0).squeeze(0), top_k_indices.squeeze(0).squeeze(0)):
          # Create a new properties dictionary to avoid modifying the original reference
          new_token = new_token.unsqueeze(0).unsqueeze(0)
          new_properties = {
              "hidden": decoder_hidden_states,  # Ensure this is properly updated
              "log_prob": properties["log_prob"] + new_prob.item(),  # Convert to Python float
              "token_list": torch.cat((properties["token_list"], new_token))  # Create a new list
          }

          # Append new beam
          new_beams.append((new_token, new_properties))

      # Sort the new beams via the log_probabilities to get the top K beams and prune the others
      new_beams = sorted(new_beams, key = lambda x: x[1]["log_prob"], reverse = True)[:k]
      curr_beams = new_beams


    #Add the curr items into final_items
    for beam in curr_beams:
      tokens = beam[0]
      properties = beam[1]
      score = properties['log_prob']
      normailizer = len(properties['token_list']) ** length_penalty_alpha
      score /= normailizer
      final_sequences.append((properties["token_list"], score))

    # Pick the token sequence with the highest normalized score
    best_sequence = max(final_sequences, key = lambda x: x[1])[0]
    best_sequence = best_sequence.squeeze(1)
    best_sequence = best_sequence.tolist()
    #Decode the best sequence to generate the best sequence
    generated_sequence = tgt_tokenizer.decode(best_sequence)
    return generated_sequence
    # END CODE

## ==== END EVALUATION PORTION

In [ ]:
## ==== BEGIN EVALUATION PORTION

# BEGIN CODE : decoding.init

# Add parameter values for your decoding strategy here. Leave empty if unused.

decoding_params = dict(
    # ADD YOUR CODE HERE
)

## ==== END EVALUATION PORTION

In [ ]:
# Please do not change anything in the following cell.

evaluator = Evaluator(src_tokenizer, tgt_tokenizer)
evaluator.set_decoding_method(rnn_better_generate)

# Evaluate enc-dec-rnn
print("EVALUATION:", "enc-dec-rnn")
evaluator.evaluate(
    os.path.join(DIRECTORY_NAME, "rnn.enc-dec"),
    validation_data['Name'], validation_data['Translation'],
    max_length = rnn_enc_dec_data_params['tgt_padding'],
    **decoding_params
)

# Evaluate enc-dec-rnn-attn
print("EVALUATION:", "enc-dec-rnn-attn")
evaluator.evaluate(
    os.path.join(DIRECTORY_NAME, "rnn.enc-dec.attn"),
    validation_data['Name'], validation_data['Translation'],
    max_length = rnn_enc_dec_attn_data_params['tgt_padding'],
    **decoding_params
)

## Congratulations!

You have reached the end of the last assignment! Hope this was a fun exercise!

Once you've run the cells above, you should be ready to submit the assignment.